In [68]:
import seaborn as sns
import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import KFold

In [76]:
x_train = pd.read_csv('./x_train.csv', index_col='ID')
y_train = pd.read_csv('./y_train.csv', index_col='ID')
#train = pd.concat([x_train, y_train], axis=1)
test = pd.read_csv('./x_test.csv', index_col='ID')
#train

In [78]:
train_fill = pd.DataFrame()
si=sorted(x_train['SUB_INDUSTRY'].unique())
for j in si:
    train1 = x_train[x_train['SUB_INDUSTRY']==j]
    train_fill=train_fill.append(train1.mask(train1==None).fillna(train1.mean()),ignore_index=False)
train_fill
train = pd.concat([train_fill, y_train], axis=1)
train

,DATE,STOCK,INDUSTRY,INDUSTRY_GROUP,SECTOR,SUB_INDUSTRY,RET_1,VOLUME_1,RET_2,VOLUME_2,...,VOLUME_16,RET_17,VOLUME_17,RET_18,VOLUME_18,RET_19,VOLUME_19,RET_20,VOLUME_20,RET
ID,,,,,,,,,,,,,,,,,,,,,
0,0,2,18,5,3,44,-0.015748,0.147931,-0.015504,0.179183,...,0.630899,0.003254,-0.379412,0.008752,-0.110597,-0.012959,0.174521,-0.002155,-0.000937,True
1,0,3,43,15,6,104,0.003984,0.108459,-0.090580,0.078135,...,0.008148,0.003774,0.035140,-0.018518,0.028545,-0.028777,-0.012058,-0.034722,0.016882,True
2,0,4,57,20,8,142,0.000440,-0.096282,-0.058896,0.084771,...,-0.010336,-0.017612,-0.354333,-0.006562,-0.519391,-0.012101,-0.356157,-0.006867,-0.308868,False
3,0,8,1,1,1,2,0.031298,-0.429540,0.007756,-0.089919,...,0.012105,0.033824,-0.290178,-0.001468,-0.663834,-0.013520,-0.562126,-0.036745,-0.631458,False
4,0,14,36,12,5,92,0.027273,-0.847155,-0.039302,-0.943033,...,-0.277083,-0.012659,0.139086,0.004237,-0.017547,0.004256,0.579510,-0.040817,0.802806,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
418590,223,5703,32,10,4,77,0.021843,-0.217823,-0.021703,-0.125333,...,-0.161543,0.007785,-0.141487,0.008205,-0.268215,-0.058448,0.057577,0.031174,-0.163350,False
418591,223,5705,35,12,5,91,-0.006920,-0.375251,0.000000,-0.029437,...,-0.955492,-0.016221,-0.171172,0.007819,1.540184,-0.015396,-0.502337,-0.011073,0.632885,False
418592,223,5709,2,1,1,5,0.021869,-0.978856,-0.005929,-1.026267,...,-0.476550,0.029714,-0.385972,-0.069148,1.780169,-0.082927,1.581453,0.098607,6.956960,True


In [82]:
# Feature engineering
new_features = []

# Conditional aggregated features
shifts = [1]  # Choose some different shifts
statistics = ['mean']  # the type of stat
gb_features = ['INDUSTRY_GROUP', 'DATE']
target_feature = 'RET'
tmp_name = '_'.join(gb_features)
for shift in shifts:
    for stat in statistics:
        name = f'{target_feature}_{shift}_{tmp_name}_{stat}'
        feat = f'{target_feature}_{shift}'
        new_features.append(name)
        for data in [train, test]:
            data[name] = data.groupby(gb_features)[feat].transform(stat)

In [83]:
data

,DATE,STOCK,INDUSTRY,INDUSTRY_GROUP,SECTOR,SUB_INDUSTRY,RET_1,VOLUME_1,RET_2,VOLUME_2,...,VOLUME_16,RET_17,VOLUME_17,RET_18,VOLUME_18,RET_19,VOLUME_19,RET_20,VOLUME_20,RET_1_INDUSTRY_GROUP_DATE_mean
ID,,,,,,,,,,,,,,,,,,,,,
418595,2,0,37,12,5,94,0.020208,0.146176,0.010059,0.224756,...,-0.301163,-0.001035,-0.416533,-0.000148,-0.004548,-0.000148,-0.161792,0.016997,-0.007221,0.005850
418596,2,1,15,4,3,37,0.009134,-0.251631,0.021913,-0.712515,...,-0.679724,-0.001544,-0.408979,0.001546,0.396372,-0.007875,-0.431760,0.001742,-0.574228,0.001747
418597,2,4,57,20,8,142,0.005008,-0.115845,0.005914,-0.107441,...,-0.451590,0.011481,-0.536967,0.009520,-0.368585,0.000000,0.022713,-0.002066,-0.207362,0.003568
418598,2,5,35,12,5,91,0.015370,-0.090295,-0.013738,0.048465,...,-0.039714,0.037018,0.665132,-0.003097,0.141991,-0.008191,-0.172382,0.005145,-0.353172,0.005850
418599,2,6,57,20,8,142,0.011419,-0.289027,0.022807,-0.262690,...,-0.565747,0.004304,-0.506291,-0.026469,-0.280666,0.010743,0.365773,-0.011134,0.933284,0.003568
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
617019,222,5707,52,18,7,122,0.010188,-0.476830,-0.006419,-0.534137,...,-0.515748,0.019115,0.361119,-0.002090,-0.132224,0.015389,-0.014298,-0.008680,0.128657,-0.003623
617020,222,5710,33,10,4,83,-0.000838,-0.063269,-0.026928,0.532781,...,-0.203097,0.032965,0.108639,0.013488,-0.458271,0.019894,-0.353293,0.013513,-0.219671,-0.001294
617021,222,5714,49,17,7,113,0.005941,-0.506350,-0.016363,-0.173802,...,0.340198,0.002121,1.087437,-0.012910,1.791362,-0.057857,6.330687,-0.000493,1.175063,-0.003437


In [84]:
target = 'RET'

n_shifts = 20  # If you don't want all the shifts to reduce noise
features = ['RET_%d' % (i + 1) for i in range(n_shifts)]
features += ['VOLUME_%d' % (i + 1) for i in range(n_shifts)]
features += new_features  # The conditional features
train[features]

,RET_1,RET_2,RET_3,RET_4,RET_5,RET_6,RET_7,RET_8,RET_9,RET_10,...,VOLUME_12,VOLUME_13,VOLUME_14,VOLUME_15,VOLUME_16,VOLUME_17,VOLUME_18,VOLUME_19,VOLUME_20,RET_1_INDUSTRY_GROUP_DATE_mean
ID,,,,,,,,,,,,,,,,,,,,,
0,-0.015748,-0.015504,0.010972,-0.014672,0.016483,0.014331,-0.017215,-0.018433,0.134146,0.182287,...,-0.816436,-0.244636,-2.042624,-0.369605,0.630899,-0.379412,-0.110597,0.174521,-0.000937,0.011825
1,0.003984,-0.090580,0.018826,-0.025540,-0.038062,-0.006873,-0.026756,0.023973,0.158731,0.045642,...,-0.043655,0.012974,-0.016922,0.037780,0.008148,0.035140,0.028545,-0.012058,0.016882,0.002373
2,0.000440,-0.058896,-0.009042,0.024852,0.009354,0.027522,-0.023047,-0.002979,-0.006342,0.055803,...,-0.187444,-0.081783,-0.356711,-0.336773,-0.010336,-0.354333,-0.519391,-0.356157,-0.308868,0.015460
3,0.031298,0.007756,-0.004632,-0.019677,0.003544,-0.002021,-0.043962,-0.004329,0.014146,-0.021946,...,-0.540519,-0.839232,-2.144763,-0.431029,0.012105,-0.290178,-0.663834,-0.562126,-0.631458,0.017253
4,0.027273,-0.039302,0.000000,0.000000,0.022321,0.018182,-0.026549,-0.017391,-0.008621,0.022026,...,-0.115091,0.271702,18.894266,0.803608,-0.277083,0.139086,-0.017547,0.579510,0.802806,0.006210
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
418590,0.021843,-0.021703,0.011141,0.021379,0.001727,0.028785,-0.020195,-0.008287,0.020077,0.002472,...,-0.321638,0.129062,0.165866,-0.085512,-0.161543,-0.141487,-0.268215,0.057577,-0.163350,0.018718
418591,-0.006920,0.000000,0.006968,0.009143,0.009226,0.012941,-0.013476,0.000000,0.007863,0.006475,...,-1.454088,-1.446695,-0.616342,-0.924640,-0.955492,-0.171172,1.540184,-0.502337,0.632885,0.016674
418592,0.021869,-0.005929,0.010989,0.006030,-0.015826,-0.008824,-0.011628,0.024928,0.001890,-0.005935,...,0.586062,2.439575,-0.422736,-0.270346,-0.476550,-0.385972,1.780169,1.581453,6.956960,0.024201


In [ ]:
X_train = train[features]
y_train = train[target]

# A quiet large number of trees with low depth to prevent overfits
rf_params = {
    'n_estimators': 128,
    'max_depth': 2**3,
    'random_state': 0,
    'n_jobs': -1
}

train_dates = train['DATE'].unique()
test_dates = test['DATE'].unique()

n_splits = 10
scores = []
models = []

splits = KFold(n_splits=n_splits, random_state=0,
               shuffle=True).split(train_dates)

for i, (local_train_dates_ids, local_test_dates_ids) in enumerate(splits):
    local_train_dates = train_dates[local_train_dates_ids]
    local_test_dates = train_dates[local_test_dates_ids]

    local_train_ids = train['DATE'].isin(local_train_dates)
    local_test_ids = train['DATE'].isin(local_test_dates)

    X_local_train = X_train.loc[local_train_ids]
    y_local_train = y_train.loc[local_train_ids]
    X_local_test = X_train.loc[local_test_ids]
    y_local_test = y_train.loc[local_test_ids]

    X_local_train = X_local_train.fillna(0)
    X_local_test = X_local_test.fillna(0)

    model = RandomForestClassifier(**rf_params)
    model.fit(X_local_train, y_local_train)

    y_local_pred = model.predict_proba(X_local_test)[:, 1]
    
    sub = train.loc[local_test_ids].copy()
    sub['pred'] = y_local_pred
    y_local_pred = sub.groupby('DATE')['pred'].transform(lambda x: x > x.median()).values

    models.append(model)
    score = accuracy_score(y_local_test, y_local_pred)
    scores.append(score)
    print(f"Fold {i+1} - Accuracy: {score* 100:.2f}%")

mean = np.mean(scores)*100
std = np.std(scores)*100
u = (mean + std)
l = (mean - std)
print(f'Accuracy: {mean:.2f}% [{l:.2f} ; {u:.2f}] (+- {std:.2f})')

Fold 1 - Accuracy: 54.71%
Fold 2 - Accuracy: 52.35%
Fold 3 - Accuracy: 52.14%
Fold 4 - Accuracy: 50.79%


In [ ]:
X_test = test[features]

rf_params['random_state'] = 0
model = RandomForestClassifier(**rf_params)
model.fit(X_train.fillna(0), y_train)
y_pred = model.predict_proba(X_test.fillna(0))[:, 1]

sub = test.copy()
sub['pred'] = y_pred
y_pred = sub.groupby('DATE')['pred'].transform(
    lambda x: x > x.median()).values

submission = pd.Series(y_pred)
submission.index = test.index
submission.name = target

submission.to_csv('./stocks.csv', index=True, header=True)